In [1]:
#!/usr/bin/env python
# coding: utf-8
import os
import pandas as pd
import xlwings as xw
#import pyodbc
import seaborn as sbrn
import sys
sys.path.append(r'C:\Funciones')
import subprocess
# Aquí pondremos la funciones SQL MS PostgreSQL y demás conexiones 
from SQL_conexions import sql_conexion
from datetime import datetime

# Creamos una función para la conexión y la ponemos en funciones

In [2]:
# Ejecutar la query
conexion = sql_conexion(server='192.168.1.91', database='Northwind', username='sa', ruta_paswd=r'C:\\Credencials')  

query = """SELECT Employees.EmployeeID,
	CONCAT(Employees.FirstName, ' ', Employees.LastName) AS NOMBRE_EMPLEADO,
	YEAR(Orders.OrderDate) AS AÑO,
	COUNT(Orders.OrderID) AS N_ORDENES,
	SUM([Order Details].UnitPrice * [Order Details].Quantity) AS IMPORTE_TOTAL
FROM Employees
	INNER JOIN Orders ON Employees.EmployeeID = Orders.EmployeeID
    INNER JOIN [Order Details] ON Orders.OrderID = [Order Details].OrderID
GROUP BY Employees.EmployeeID, Employees.FirstName, Employees.LastName, YEAR(Orders.OrderDate)"""
df = pd.read_sql(query,conexion)
# Cerramos la conexión
conexion.close()


Conexión exitosa


C:\Users\JuanCarlosRomero\AppData\Local\Temp\ipykernel_11024\86498618.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query,conexion)


In [3]:
df

,EmployeeID,NOMBRE_EMPLEADO,AÑO,N_ORDENES,IMPORTE_TOTAL
0,3,Janet Leverling,1996,43,19231.80
1,9,Anne Dodsworth,1998,46,42020.75
2,6,Michael Suyama,1997,86,45992.00
3,7,Robert King,1998,60,56502.05
4,1,Nancy Davolio,1997,156,97533.58
5,5,Steven Buchanan,1996,27,21965.20
6,2,Andrew Fuller,1998,99,79955.96
7,6,Michael Suyama,1998,44,14475.00
8,4,Margaret Peacock,1998,105,57594.95
9,8,Laura Callahan,1997,124,59776.52


In [4]:
#Dentro de excel editamos manualmente el reporte a automatizar, una tabla dinámica y una gráfica.


# Creamos una ruta de reportes
ruta_report = r'C:\Reportes' 
ruta_actual = os.getcwd()


In [14]:
# Ya que tenemos la automatización comenzamos a escribir el código para generar el REPORTE
# Exportamos la DF a excel en un reporte temporal
df.to_excel('REPORTE_T.xlsx', sheet_name='DATOS', index=False)
# Ver el proceso en el caso de pruebas va True de Xlwings
app = xw.App(visible=False)
#wb_0 = app.books.add()
wb = xw.Book('REPORTE_T.xlsx')
wb_2 = xw.Book('TEMPLATE_M.xlsm')
book1 = xw.books["Book1"]
# Aprevechamos el libro temporal para escribir datos como nombre, fecha, rutas de origen y destino
book1.sheets(1)['A1'].value = "REPORTE"
book1.sheets(1)['A2'].value = datetime.today().strftime('%d_%m_%Y_%H_%M')
book1.sheets(1)['A3'].value = ruta_report
book1.sheets(1)['A4'].value = ruta_actual
# Con este comando llamamos al Template de macros
CREAR_TABLA = wb_2.macro('CrearTablaPivot')
GUARDAR = wb_2.macro('GUARDAR_7')
# Llamar a las macros
CREAR_TABLA()
GUARDAR()
# Cerramos los archivos
book1.close()
wb_2.close()
wb.close()
app.quit()
# Guardamos el reporte
ruta3 = ruta_actual + "\\" "REPORTE_T.xlsx"
os.remove(ruta3)

# Cerramos la app 
xw.App().quit()
# Terminamos procesos hijos de jexcel
subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])
subprocess.call(["taskkill", "/f", "/im", "EXCEL.EXE"])

1